In [1]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
dfFac = pd.read_sql("SELECT Game_Id, Date, Description FROM PBP WHERE Event == 'FAC' AND Period == '1' AND Seconds_Elapsed == '0'", conn)
dfFac[['gagneMAJ', 'Reste']] = dfFac.Description.str.split(" won",expand=True,)
dfFac[['Année', 'Reste', 'Reste2']] = dfFac.Date.str.split("-",expand=True)
dfFac = dfFac.drop(["Description", "Reste2"], axis=1)
s = pd.to_numeric(dfFac["Année"])
dfFac["Année"] = s
r = pd.to_numeric(dfFac["Reste"])
dfFac["Reste"] = r
dfFac["debutSaison"] = r.ge(9)
dfFac["SaisonTempo"] = s * dfFac["debutSaison"]
st = dfFac["SaisonTempo"]
dfFac["finSaison"] = st.eq(0)
dfFac["SaisonTempo2"] = (s - 1) * dfFac["finSaison"]
dfFac["Saison"] = dfFac["SaisonTempo"] + dfFac["SaisonTempo2"]
dfFac["Saison"] = dfFac["Saison"].apply(str)
dfFac = dfFac.drop(["Reste", "Année", "debutSaison", "SaisonTempo", "finSaison", "SaisonTempo2"], axis=1)
dfFac["IdMatch"] = dfFac["Saison"] + "0" + dfFac["Game_Id"]

df = pd.read_csv("C:\dev\pool-de-hockey\src\data\scoreMatchs.csv", sep=";")
df[['Année', 'Reste', 'Reste2']] = df.Date.str.split("-",expand=True)
df = df.drop(["Reste2"], axis=1)
s = pd.to_numeric(df["Année"])
df["Année"] = s
r = pd.to_numeric(df["Reste"])
df["Reste"] = r
df["debutSaison"] = r.ge(9)
df["SaisonTempo"] = s * df["debutSaison"]
st = df["SaisonTempo"]
df["finSaison"] = st.eq(0)
df["SaisonTempo2"] = (s - 1) * df["finSaison"]
df["Saison"] = df["SaisonTempo"] + df["SaisonTempo2"]
df["Saison"] = df["Saison"].apply(str)
df["Game_Id"] = df["Game_Id"].apply(str)
df = df.drop(["Reste", "Année", "debutSaison", "SaisonTempo", "finSaison", "SaisonTempo2"], axis=1)
df["IdMatch"] = df["Saison"] + "0" + df["Game_Id"]

dfFinale = dfFac.merge(df, on = "IdMatch")
dfFinale = dfFinale[["Date_x", "gagneMAJ", "gagnant"]]
#dfFinale.to_csv("C:\dev\pool-de-hockey\src\data\corr.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\dev\\pool-de-hockey\\src\\data\\scoreMatchs.csv'

In [4]:
#dfVic = pd.read_sql("SELECT Game_Id, Date, Event, Away_Team, Home_Team, Away_Score, Home_Score FROM PBP WHERE Event == 'GEND'", conn)
#dfVic.to_csv("C:\dev\pool-de-hockey\src\data\scoreMatchs.csv")

In [1]:
from math import *
import math
from pandas import DataFrame


def loiBinom(p, n, x):
    pre = math.factorial(n)/(math.factorial(x)*math.factorial(n - x))
    mid = pow(p, x)
    end = pow((1 - p), (n - x))
    return pre * mid * end

def probGagnerPari(p, n, gagnerAbs=False):
    if gagnerAbs:
        mi = floor(n/2)
        mi = mi + 1
    else:
        mi = ceil(n/2)
    tot = 0
    for i in range (mi, n+1):
        f = loiBinom(p, n, i)
        tot = tot + f
    return tot

def mettreDf(p, n, gagnerAbs=False):
    lis = list(range(1, n+1))
    df = DataFrame (lis, columns=["n"])
    vec = []
    p = 0.5075
    for i in lis:
        probGagner = probGagnerPari(p, i, gagnerAbs)
        vec.append(probGagner)
    df["probGagner"] = vec
    return df

In [16]:
lis = []
df = mettreDf(0.5075, 1001, True)
df = df.sort_values(by="probGagner", ascending=False)
df

,n,probGagner
1,2,0.757444
3,4,0.698665
999,1000,0.693609
997,998,0.693454
995,996,0.693298
...,...,...
8,9,0.518451
6,7,0.516403
4,5,0.514060
2,3,0.511249


In [2]:
loiBinom(0.2, 100, 24)

0.057733990380810424

In [19]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
master_buts = pd.read_sql("SELECT Game_Id, Date, Event, Ev_team, Away_Team, Home_Team, Away_Goalie, Home_Goalie FROM PBP WHERE Event == 'GOAL'", conn)

#ajout colonne comptéSur
master_buts["comptéSur"] = np.where(master_buts["Ev_Team"]==master_buts["Away_Team"], master_buts["Home_Goalie"], master_buts["Away_Goalie"])

uniq = pd.unique(master_buts[['Away_Goalie', 'Home_Goalie']].values.ravel('K'))
uniqueGar = uniq.tolist()
for i in range(0,136):
    gardien = uniqueGar[i]
    sql = "SELECT * FROM SHITFS WHERE GARDIEN IS GARDIEN"

## rendu à la requete sql 

['CRAIG ANDERSON',
 'BRADEN HOLTBY',
 'JACOB MARKSTROM',
 'MARTIN JONES',
 'SERGEI BOBROVSKY',
 None,
 'CONNOR HELLEBUYCK',
 'CARTER HUTTON',
 'CAREY PRICE',
 'DEVAN DUBNYK',
 'TUUKKA RASK',
 'DAVID RITTICH',
 'DARCY KUEMPER',
 'COREY CRAWFORD',
 'LAURENT BROSSOIT',
 'ILYA SAMSONOV',
 'FREDERIK ANDERSEN',
 'MARC-ANDRE FLEURY',
 'MACKENZIE BLACKWOOD',
 'ALEXANDAR GEORGIEV',
 'ANDREI VASILEVSKIY',
 'ELVIS MERZLIKINS',
 'JAMES REIMER',
 'BEN BISHOP',
 'JIMMY HOWARD',
 'JAROSLAV HALAK',
 'AARON DELL',
 'JONATHAN QUICK',
 'CURTIS MCELHINNEY',
 'ANTON KHUDOBIN',
 'JORDAN BINNINGTON',
 'LINUS ULLMARK',
 'MIKKO KOSKINEN',
 'JOHN GIBSON',
 'JACK CAMPBELL',
 'KEITH KINKAID',
 'CORY SCHNEIDER',
 'JONATHAN BERNIER',
 'JAKE ALLEN',
 'MALCOLM SUBBAN',
 'RYAN MILLER',
 'THOMAS GREISS',
 'MIKE SMITH',
 'JUUSE SAROS',
 'SAM MONTEMBEAULT',
 'JOONAS KORPISALO',
 'MATT MURRAY',
 'ANTTI RAANTA',
 'CARTER HART',
 'TRISTAN JARRY',
 'CAM TALBOT',
 'ALEX STALOCK',
 'PHILIPP GRUBAUER',
 'BRIAN ELLIOTT',
 'PEKKA